In [2]:
# Задание 1
import requests
from bs4 import BeautifulSoup
import pandas as pd


url = 'https://habr.com/ru/all/'

KEYWORDS = ['python', 'парсинг', 'frida', 'otus', 'angular', 'dart', 'линус']

df = pd.DataFrame()

def search_words(words: str) -> bool:
    words = words.lower()
    if any([desired in words for desired in KEYWORDS]):
        return True
    return False


def post_text(url_post: str) -> str:
    resp = requests.get(url_post)
    if resp.status_code == 200:
        soup = BeautifulSoup(resp.text, 'html.parser')
        return soup.find('div', class_='post__text').text, soup.find('span', class_='post__time').get('data-time_published')
    return


resp = requests.get(url)
if resp.status_code == 200:
    soup = BeautifulSoup(resp.text, 'html.parser')
    posts = soup.find_all('article', class_='post')

    for post in posts:
        search_id = False
        post_id = post.parent.attrs.get('id')
    # если идентификатор не найден, это что-то странное, пропускаем
        if not post_id:
            continue
        post_id = int(post_id.split('_')[-1])

        # ищем в заголовке
        titles = post.find_all('a', class_='post__title_link')
        for title in titles:
            if search_words(title.text):
                print(post_id, 'title')
                search_id = True
                break

        # ищем в хабах
        if not search_id:
            hubs = post.find_all('ul', class_='post__hubs')
            hub_lower = ''
            for hub in hubs:
                hub_lower += hub.text   
            search_id = search_words(hub_lower)

                
        # ищем в тексте
        if not search_id:
            texts = post.find_all('div', class_='post__text')
            for text in texts:
                if search_words(text.text):
                    search_id = True
                    break

        # ищем в статье
        if not search_id:
            title_element = post.find('a', class_='post__title_link')
            url_post = title_element.attrs.get('href')
            art_text, _ = post_text(url_post)    
            search_id = search_words(art_text)

        if search_id:
            title = post.find('a', class_='post__title_link')
            url_post = title.attrs.get('href')
            text, date = post_text(url_post)
            row = {'date': pd.to_datetime(date).date(), 'title': title.text, 'url': url_post, 'text': text}
            df = pd.concat([df, pd.DataFrame([row])])

df

,date,title,url,text
0,2020-12-09,Umka и трактор: первый опыт практического прим...,https://habr.com/ru/post/532018/,С весны нынешнего года я разрабатываю статичес...
0,2020-12-09,MikroTik Скрипт: Уведомление о успешном входе ...,https://habr.com/ru/post/532086/,Статья будет больше интересна специалистам исп...


In [ ]:
# Задание 2